<a href="https://colab.research.google.com/github/hualinyc/my-first-blog/blob/master/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [1]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-thinking-exp-1219' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IldoYXQgaXMgYmlnZ2VyLCA5Ljkgb3IgOS4xMT8ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiIxLiAqKklkZW50aWZ5IHRoZSBjb3JlIHF1ZXN0aW9uOioqIFRoZSBxdWVzdGlvbiBhc2tzIGZvciBhIGNvbXBhcmlzb24gYmV0d2VlbiB0d28gZGVjaW1hbCBudW1iZXJzOiA5LjkgYW5kIDkuMTEuXFxuXFxuMi4gKipGb2N1cyBvbiB0aGUgZGlmZmVyaW5nIHBhcnRzOioqICBCb3RoIG51bWJlcnMgc2hhcmUgdGhlIHdob2xlIG51bWJlciBwYXJ0ICg5KS4gVGhlIGRpZmZlcmVuY2UgbGllcyBpbiB0aGUgZGVjaW1hbCBwYXJ0LlxcblxcbjMuICoqQ29tcGFyZSB0aGUgdGVudGhzIHBsYWNlOioqIFRoZSBmaXJzdCBkZWNpbWFsIHBsYWNlICh0ZW50aHMpIGlzIDkgaW4gOS45IGFuZCAxIGluIDkuMTEuIFNpbmNlIDkgaXMgZ3JlYXRlciB0aGFuIDEsIG9uZSBtaWdodCBpbml0aWFsbHkgdGhpbmsgOS45IGlzIGxhcmdlci5cXG5cXG40LiAqKlJlYWxpemUgdGhlIGltcG9ydGFuY2Ugb2YgcGxhY2UgdmFsdWU6KiogVGhlIG51bWJlciBvZiBkZWNpbWFsIHBsYWNlcyBtYXR0ZXJzLiBUbyBtYWtlIGEgZmFpciBjb21wYXJpc29uIG9mIHRoZSBkZWNpbWFsIHBhcnRzLCB3ZSBuZWVkIHRoZW0gdG8gaGF2ZSB0aGUgc2FtZSBudW1iZXIgb2YgZGlnaXRzLlxcblxcbjUuICoqQWRkIGEgdHJhaWxpbmcgemVybzoqKiAgV2UgY2FuIHJld3JpdGUgOS45IGFzIDkuOTAgd2l0aG91dCBjaGFuZ2luZyBpdHMgdmFsdWUuIE5vdyB3ZSBoYXZlIDkuOTAgYW5kIDkuMTEuXFxuXFxuNi4gKipDb21wYXJlIHRoZSBkZWNpbWFsIHBhcnRzIGRpcmVjdGx5OioqICBOb3cgd2UgY29tcGFyZSA5MCBhbmQgMTEuIENsZWFybHksIDkwIGlzIGdyZWF0ZXIgdGhhbiAxMS5cXG5cXG43LiAqKkNvbmNsdWRlOioqIFNpbmNlIHRoZSBkZWNpbWFsIHBhcnQgb2YgOS45MCBpcyBsYXJnZXIgdGhhbiB0aGUgZGVjaW1hbCBwYXJ0IG9mIDkuMTEsIGFuZCB0aGUgd2hvbGUgbnVtYmVyIHBhcnRzIGFyZSB0aGUgc2FtZSwgOS45IGlzIGJpZ2dlciB0aGFuIDkuMTEuIn0seyJ0ZXh0IjoiVG8gZGV0ZXJtaW5lIHdoaWNoIG51bWJlciBpcyBiaWdnZXIsIHdlIGNvbXBhcmUgdGhlaXIgdmFsdWVzLlxcblxcbkJvdGggbnVtYmVycyBoYXZlIHRoZSBzYW1lIHdob2xlIG51bWJlciBwYXJ0LCB3aGljaCBpcyA5LiBTbywgd2UgbmVlZCB0byBjb21wYXJlIHRoZSBkZWNpbWFsIHBhcnRzLlxcblxcbkZvciA5LjksIHRoZSBkZWNpbWFsIHBhcnQgaXMgOS4gV2UgY2FuIHRoaW5rIG9mIHRoaXMgYXMgOSB0ZW50aHMuXFxuRm9yIDkuMTEsIHRoZSBkZWNpbWFsIHBhcnQgaXMgMTEuIFdlIGNhbiB0aGluayBvZiB0aGlzIGFzIDEgdGVudGggYW5kIDEgaHVuZHJlZHRoLlxcblxcblRvIG1ha2UgdGhlIGNvbXBhcmlzb24gZWFzaWVyLCB3ZSBjYW4gd3JpdGUgOS45IHdpdGggdHdvIGRlY2ltYWwgcGxhY2VzIGJ5IGFkZGluZyBhIHRyYWlsaW5nIHplcm86IDkuOTAuXFxuXFxuTm93IHdlIGNvbXBhcmUgOS45MCBhbmQgOS4xMS4gVGhlIHdob2xlIG51bWJlciBwYXJ0IGlzIHRoZSBzYW1lLiBXZSBjb21wYXJlIHRoZSBkZWNpbWFsIHBhcnRzOlxcblxcbiogOTAgKGZyb20gOS45MClcXG4qIDExIChmcm9tIDkuMTEpXFxuXFxuU2luY2UgOTAgaXMgZ3JlYXRlciB0aGFuIDExLCA5LjkwIGlzIGdyZWF0ZXIgdGhhbiA5LjExLlxcblxcblRoZXJlZm9yZSwgKio5LjkgaXMgYmlnZ2VyIHRoYW4gOS4xMSoqLlxcbiJ9XX0seyJyb2xlIjoidXNlciIsInBhcnRzIjpbeyJ0ZXh0Ijoid2hpY2ggaXMgYmlnZ2VyLCAgMC4wIG9yIDAuMDA/ICJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUncyBteSB0aGlua2luZyBwcm9jZXNzIHRvIGRldGVybWluZSBpZiAwLjAgaXMgYmlnZ2VyIHRoYW4gMC4wMDpcXG5cXG4xLiAqKlVuZGVyc3RhbmQgdGhlIHF1ZXN0aW9uOioqIFRoZSBxdWVzdGlvbiBhc2tzIGZvciBhIGNvbXBhcmlzb24gb2YgdHdvIGRlY2ltYWwgbnVtYmVyczogMC4wIGFuZCAwLjAwLiBUaGUgZ29hbCBpcyB0byBpZGVudGlmeSB3aGljaCBpcyBcXFwiYmlnZ2VyXFxcIi5cXG5cXG4yLiAqKkluaXRpYWwgT2JzZXJ2YXRpb246KiogIEJvdGggbnVtYmVycyBhcHBlYXIgdG8gcmVwcmVzZW50IG5vdGhpbmcuIFRoaXMgc3VnZ2VzdHMgdGhleSBtaWdodCBiZSBlcXVhbC5cXG5cXG4zLiAqKkNvbnNpZGVyIHRoZSBwbGFjZSB2YWx1ZXM6KiogIEJyZWFrIGRvd24gdGhlIG51bWJlcnMgYmFzZWQgb24gdGhlaXIgcGxhY2UgdmFsdWVzOlxcblxcbiAgICogKiowLjA6KiogIFRoZSBmaXJzdCAwIGlzIGluIHRoZSBvbmVzIHBsYWNlLiBUaGUgc2Vjb25kIDAgaXMgaW4gdGhlIHRlbnRocyBwbGFjZS4gVGhpcyBtZWFucyB6ZXJvIG9uZXMgYW5kIHplcm8gdGVudGhzLlxcblxcbiAgICogKiowLjAwOioqIFRoZSBmaXJzdCAwIGlzIGluIHRoZSBvbmVzIHBsYWNlLiBUaGUgc2Vjb25kIDAgaXMgaW4gdGhlIHRlbnRocyBwbGFjZS4gVGhlIHRoaXJkIDAgaXMgaW4gdGhlIGh1bmRyZWR0aHMgcGxhY2UuIFRoaXMgbWVhbnMgemVybyBvbmVzLCB6ZXJvIHRlbnRocywgYW5kIHplcm8gaHVuZHJlZHRocy5cXG5cXG40LiAqKk51bWVyaWNhbCBSZXByZXNlbnRhdGlvbjoqKlxcblxcbiAgICogMC4wIGNhbiBiZSB3cml0dGVuIGFzIDAgKyAoMC8xMCkgPSAwXFxuICAgKiAwLjAwIGNhbiBiZSB3cml0dGVuIGFzIDAgKyAoMC8xMCkgKyAoMC8xMDApID0gMFxcblxcbjUuICoqQ29tcGFyaXNvbjoqKiBCb3RoIG51bWJlcnMgZXZhbHVhdGUgdG8gdGhlIHNhbWUgdmFsdWU6IDAuXFxuXFxuNi4gKipGb3JtYWwgQ29tcGFyaXNvbjoqKiAgV2hlbiBjb21wYXJpbmcgbnVtYmVycywgd2UgbG9vayBmb3IgZGlmZmVyZW5jZXMgaW4gdGhlaXIgdmFsdWVzLiBTaW5jZSBib3RoIHJlcHJlc2VudCB6ZXJvLCB0aGVyZSBpcyBubyBkaWZmZXJlbmNlLlxcblxcbjcuICoqQ29uY2x1c2lvbjoqKiAgVGhlcmVmb3JlLCAwLjAgYW5kIDAuMDAgYXJlIGVxdWFsIGluIHZhbHVlLiBOZWl0aGVyIGlzIFxcXCJiaWdnZXJcXFwiIHRoYW4gdGhlIG90aGVyLlxcblxcbjguICoqUmVmaW5lIHRoZSBhbnN3ZXIgKGFkZHJlc3MgcG90ZW50aWFsIG51YW5jZXMpOioqICBXaGlsZSB0aGV5IGFyZSBudW1lcmljYWxseSBlcXVhbCwgaXQncyB3b3J0aCBub3RpbmcgaG93IHRoZXkgKm1pZ2h0KiBiZSBwZXJjZWl2ZWQgaW4gc29tZSBjb250ZXh0czpcXG5cXG4gICAqICoqUHJlY2lzaW9uOioqICBJbiBzb21lIHNjaWVudGlmaWMgb3IgY29tcHV0YXRpb25hbCBjb250ZXh0cywgMC4wMCBtaWdodCBpbXBseSBhIGhpZ2hlciBsZXZlbCBvZiBwcmVjaXNpb24gb3IgbWVhc3VyZW1lbnQgdGhhbiAwLjAuIEhvd2V2ZXIsIGluIHRlcm1zIG9mIHRoZSBhY3R1YWwgbnVtZXJpY2FsIHZhbHVlLCB0aGV5IGFyZSB0aGUgc2FtZS5cXG5cXG4gICAqICoqVHJhaWxpbmcgWmVyb3M6KiogVHJhaWxpbmcgemVyb3MgYWZ0ZXIgYSBkZWNpbWFsIHBvaW50IGRvIG5vdCBjaGFuZ2UgdGhlIHZhbHVlIG9mIHRoZSBudW1iZXIuIFRoZXkgY2FuIHNvbWV0aW1lcyBpbmRpY2F0ZSBwcmVjaXNpb24sIGJ1dCBpbiB0ZXJtcyBvZiBtYWduaXR1ZGUsIHRoZXkgYXJlIGVxdWl2YWxlbnQuXFxuXFxuVGhlcmVmb3JlLCB0aGUgbW9zdCBhY2N1cmF0ZSBhbmQgZGlyZWN0IGFuc3dlciBpcyB0aGF0IHRoZXkgYXJlIGVxdWFsLiJ9LHsidGV4dCI6IjAuMCBhbmQgMC4wMCBhcmUgKiplcXVhbCoqLlxcblxcbkhlcmUncyB3aHk6XFxuXFxuKiAqKlZhbHVlOioqIEJvdGggcmVwcmVzZW50IHplcm8gYW5kIHplcm8gdGVudGhzLCBhbmQgaW4gdGhlIGNhc2Ugb2YgMC4wMCwgemVybyBodW5kcmVkdGhzIGFzIHdlbGwuXFxuKiAqKlRyYWlsaW5nIFplcm9zOioqIFRyYWlsaW5nIHplcm9zIGFmdGVyIHRoZSBkZWNpbWFsIHBvaW50IGRvIG5vdCBjaGFuZ2UgdGhlIHZhbHVlIG9mIGEgbnVtYmVyLiAwLjAwIGlzIHNpbXBseSBhIG1vcmUgcHJlY2lzZSB3YXkgb2Ygd3JpdGluZyAwLjAsIGltcGx5aW5nIG1lYXN1cmVtZW50IHRvIHRoZSBodW5kcmVkdGhzIHBsYWNlLlxcblxcblRoaW5rIG9mIGl0IGxpa2UgdGhpczpcXG5cXG4qIDAuMCBtZWFucyB6ZXJvLlxcbiogMC4wMCBhbHNvIG1lYW5zIHplcm8uXFxuXFxuVGhlcmVmb3JlLCBuZWl0aGVyIGlzIGJpZ2dlciB0aGFuIHRoZSBvdGhlcjsgdGhleSBhcmUgdGhlIHNhbWUuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "What is bigger, 9.9 or 9.11?"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "1. **Identify the core question:** The question asks for a comparison between two decimal numbers: 9.9 and 9.11.\\n\\n2. **Focus on the differing parts:**  Both numbers share the whole number part (9). The difference lies in the decimal part.\\n\\n3. **Compare the tenths place:** The first decimal place (tenths) is 9 in 9.9 and 1 in 9.11. Since 9 is greater than 1, one might initially think 9.9 is larger.\\n\\n4. **Realize the importance of place value:** The number of decimal places matters. To make a fair comparison of the decimal parts, we need them to have the same number of digits.\\n\\n5. **Add a trailing zero:**  We can rewrite 9.9 as 9.90 without changing its value. Now we have 9.90 and 9.11.\\n\\n6. **Compare the decimal parts directly:**  N

## Call `generate_content`

In [6]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

contents = [{'role': 'user', 'parts': [{'text': 'What is bigger, 99.9 or 9.11?'}]}]
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

print("i'm here")
print("contents=", contents)
print("response=", response)
print("response text=", response.text, "\n********end****")
display(Markdown(response.text))

i'm here
contents= [{'role': 'user', 'parts': [{'text': 'What is bigger, 99.9 or 9.11?'}]}]
response= response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "1. **Analyze the question:** The question asks which of two numbers, 99.9 and 9.11, is bigger. This is a straightforward comparison of numerical values.\n\n2. **Identify the numbers:** The two numbers are 99.9 and 9.11.\n\n3. **Compare the whole number parts:**  The whole number part of 99.9 is 99. The whole number part of 9.11 is 9. Since 99 is greater than 9, we can immediately conclude that 99.9 is bigger than 9.11.\n\n4. **Consider the decimal parts (for completeness/confirmation):** While the whole number comparison is sufficient, we can also look at the decimal parts. Even if the whole number parts were the same, we would compare the tenths place, then

1. **Analyze the question:** The question asks which of two numbers, 99.9 and 9.11, is bigger. This is a straightforward comparison of numerical values.

2. **Identify the numbers:** The two numbers are 99.9 and 9.11.

3. **Compare the whole number parts:**  The whole number part of 99.9 is 99. The whole number part of 9.11 is 9. Since 99 is greater than 9, we can immediately conclude that 99.9 is bigger than 9.11.

4. **Consider the decimal parts (for completeness/confirmation):** While the whole number comparison is sufficient, we can also look at the decimal parts. Even if the whole number parts were the same, we would compare the tenths place, then the hundredths place, and so on. In this case, the decimal parts are irrelevant to the comparison because the whole number parts differ.

5. **State the conclusion:** Based on the comparison, 99.9 is bigger than 9.11.
**99.9** is bigger than 9.11.

Think of it this way:

* 99.9 is almost 100.
* 9.11 is a little more than 9.

100 is much larger than 9.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [7]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

What is bigger, 9.9 or 9.11? 

-------------------------------------------------------------------------------- 

Role: model 

1. **Identify the core question:** The question asks for a comparison between two decimal numbers: 9.9 and 9.11.\n\n2. **Focus on the differing parts:**  Both numbers share the whole number part (9). The difference lies in the decimal part.\n\n3. **Compare the tenths place:** The first decimal place (tenths) is 9 in 9.9 and 1 in 9.11. Since 9 is greater than 1, one might initially think 9.9 is larger.\n\n4. **Realize the importance of place value:** The number of decimal places matters. To make a fair comparison of the decimal parts, we need them to have the same number of digits.\n\n5. **Add a trailing zero:**  We can rewrite 9.9 as 9.90 without changing its value. Now we have 9.90 and 9.11.\n\n6. **Compare the decimal parts directly:**  Now we compare 90 and 11. Clearly, 90 is greater than 11.\n\n7. **Conclude:** Since the decimal part of 9.90 i